In [1]:
import pandas as pd
import re
import random

random.seed(42)
from tqdm.auto import tqdm
tqdm.pandas()

/share/u/arunas/miniconda3/envs/broca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Genders for subjects based on italian

In [2]:
en_it = {
    "lemonade": "limonata",
    "espresso": "espresso",
    "milkshake": "frappè",
    "smoothie": "frullato",
    "beverage": "bevanda",
    "tonic": "tonico",
    "water": "acqua",
    "coffee": "caffè",
    "tea": "tè",
    "juice": "succo",
    "pear": "pera",
    "pole": "palo",
    "writer": "scrittore",
    "writers": "scrittori",
    "banana": "banana",
    "biscuit": "biscotto",
    "book": "libro",
    "bottle": "bottiglia",
    "box": "scatola",
    "boy": "ragazzo",
    "boys": "ragazzi",
    "lightbulb": "lampadina",
    "sideboard": "credenza",
    "hat": "cappello",
    "cat": "gatto",
    "cats": "gatti",
    "mount": "monte",
    "wall": "muro",
    "chapter": "capitolo",
    "chalk": "gesso",
    "cup": "tazza",
    "cucumber": "cetriolo",
    "dog": "cane",
    "dogs": "cani",
    "oratory": "oratorio",
    "fish": "pesce",
    "fruit": "frutta",
    "girl": "ragazza",
    "girls": "ragazze",
    "hill": "collina",
    "man": "uomo",
    "men": "uomini",
    "meal": "pasto",
    "mountain": "montagna",
    "mouse": "topo",
    "newspaper": "giornale",
    "pizza": "pizza",
    "poet": "poeta",
    "poets": "poeti",
    "poem": "poema",
    "rock": "roccia",
    "roof": "tetto",
    "orator": "oratore",
    "orators": "oratori",
    "ladder": "scala",
    "story": "storia",
    "teacher": "insegnante",
    "teachers": "insegnanti",
    "toy": "giocattolo",
    "tree": "albero",
    "woman": "donna",
    "women": "donne",
    "author": "autore",
    "authors": "autori",
    "actress": "attrice",
    "teacher": "maestra",
    "doctor": "dottoressa",
    "doctors": "dottoresse",
    "lawyer": "avvocata",
    "architect": "architetta",
    "professor": "professoressa",
    "writers": "scrittrici",
    "cook": "cuoca",
    "letter": "lettera",
    "guide": "guida",
    "postcard": "cartolina",
    "novella": "novella",
    "page": "pagina",
    "poetry": "poesia",
    "magazine": "rivista",
    "actresses": "attrici",
    "teachers": "maestre",
    "doctors": "dottoresse", 
    "lawyers": "avvocate",
    "architects": "architette",
    "professors": "professoresse",
    "cooks": "cuoche",
    "letters": "lettere",
    "guides": "guide",
    "postcards": "cartoline",
    "novellas": "novelle",
    "pages": "pagine",
    "poems": "poesie",
    "magazines": "riviste"
}

pastTense = {
    'hits': 'hit',
    'kicks': 'kicked',
    'touches': 'touched',
    'pushes': 'pushed',
    'drinks': 'drunk',
    'eats': 'ate',
    'reads': 'read',
    'hit': 'hit',
    'kick': 'kicked',
    'touch': 'touched',
    'push': 'pushed',
    'drink': 'drunk',
    'eat': 'ate',
    'read': 'read'
}

infinitive = {
    'hits': 'to hit',
    'kicks': 'to kick',
    'touches': 'to touch',
    'pushes': 'to push',
    'drinks': 'to drink',
    'eats': 'to eat',
    'reads': 'to read',
    'hit': 'to hit',
    'kick': 'to kick',
    'touch': 'to touch',
    'push': 'to push',
    'drink': 'to drink',
    'eat': 'to eat',
    'read': 'to read'
}

pluralObjects = {
    'fish': 'fish',
    'mouse': 'mice',
    'bottle': 'bottles',
    'newspaper': 'newspapers',
    'chalk': 'chalks',
    'box': 'boxes',
    'cap': 'caps',
    'bulb': 'bulbs',
    'cup': 'cups',
    'toy': 'toys',
    'staircase': 'staircases',
    'rock': 'rocks',
    'hill': 'hills',
    'mountain': 'mountains',
    'roof': 'roofs',
    'tree': 'trees',
    'biscuit': 'biscuits',
    'banana': 'bananas',
    'pear': 'pears',
    'meal': 'meals',
    'fruit': 'fruits',
    'cucumber': 'cucumbers',
    'pizza': 'pizzas',
    'book': 'books',
    'poem': 'poems',
    'story': 'stories',
    'chapter': 'chapters'
}

passiveSeed = {
    'hits': 'hit',
    'kicks': 'kicked',
    'touches': 'touched',
    'pushes': 'pushed',
    'drinks': 'drunk',
    'eats': 'eaten',
    'reads': 'read',
    'hit': 'hit',
    'kick': 'kicked',
    'touch': 'touched',
    'push': 'pushed',
    'drink': 'drunk',
    'eat': 'eaten',
    'read': 'read'
}


In [3]:
it_genderDict = {
    "limonata": [ "la", "una"],
    "espresso": [ "il", "un"],
    "frappè": [ "il", "un"],
    "frullato": [ "il", "un"],
    "bevanda": [ "la", "una"],
    "tonico": [ "il", "un"],
    "acqua": [ "l'", "un'"],
    "caffè": [ "il", "un"],
    "tè": [ "il", "un"],
    "succo": [ "il", "un"],
    "caffè": ["il", "un"],
    "tonico": ["il", "un"],
    "tè": ["il", "un"],
    "succo": ["il", "un"],
    "acqua": ["la", "una"],
    "pera": ["la", "una"],
    "palo": ["il", "un"],
    "scrittore": ["lo", "uno"],
    "scrittori": ["gli"],
    "banana": ["la", "una"],
    "biscotto": ["il", "un"],
    "libro": ["il", "un"],
    "bottiglia": ["la", "una"],
    "scatola": ["la", "una"],
    "ragazzo": ["il", "un"],
    "ragazzi": ["i"],
    "lampadina": ["la", "una"],
    "credenza": ["la", "una"],
    "cappello": ["il", "un"],
    "gatto": ["il", "un"],
    "gatti": ["i"],
    "monte": ["il", "un"],
    "muro": ["il", "un"],
    "capitolo": ["il", "un"],
    "gesso": ["il", "un"],
    "tazza": ["la", "una"],
    "cetriolo": ["il", "un"],
    "cane": ["il", "un"],
    "cani": ["i"],
    "oratorio": ["il", "un"],
    "pesce": ["il", "un"],
    "frutta": ["la", "una"],
    "ragazza": ["la", "una"],
    "ragazze": ["le"],
    "collina": ["la", "una"],
    "uomo": ["l'", "un"],
    "uomini": ["gli"],
    "pasto": ["il", "un"],
    "montagna": ["la", "una"],
    "topo": ["il", "un"],
    "giornale": ["il", "un"],
    "pizza": ["la", "una"],
    "poeta": ["il", "un"],
    "poeti": ["i"],
    "poema": ["il", "un"],
    "roccia": ["la", "una"],
    "tetto": ["il", "un"],
    "oratore": ["l'", "un'"],
    "oratori": ["gli"],
    "scala": ["la", "una"],
    "storia": ["la", "una"],
    "insegnante": ["l'", "un'"],
    "insegnanti": ["gli"],
    "giocattolo": ["il", "un"],
    "albero": ["l'", "un"],
    "donna": ["la", "una"],
    "donne": ["le"],
    "autore": ["l'", "un'"],
    "autori": ["gli"],
    "attrice": ["l'", "un'"], 
    "maestra": ["la", "una"], 
    "dottoressa": ["la", "una"], 
    "dottoresse": ["le"],
    "avvocata": ["l'", "un'"], 
    "architetta": ["l'", "un'"], 
    "professoressa": ["la", "una"], 
    "scrittrice": ["la", "una"], 
    "cuoca": ["la", "una"], 
    "lettera": ["la", "una"], 
    "guida": ["la", "una"], 
    "cartolina": ["la", "una"], 
    "novella": ["la", "una"], 
    "pagina": ["la", "una"], 
    "poesia": ["la", "una"],
    "rivista": ["la", "una"],
    "attrici": ["le"], 
    "maestre": ["le"], 
    "dottoressi": ["le"], 
    "avvocate": ["l'"], 
    "architette": ["l'"], 
    "professoresse": ["le"], 
    "scrittrici": ["le"], 
    "cuoche": ["le"], 
    "lettere": ["le"], 
    "guide": ["le"], 
    "cartoline": ["le"], 
    "novelle": ["le"], 
    "pagine": ["le"], 
    "poesie": ["le"],
    "riviste": ["le"]
}

it_infinitive = {
    'colpisce' : 'colpire',
    'legge': 'leggere',
    'beve': 'bere',
    'mangia': 'mangiare',
    'tocca': 'toccare',
    'spinge' : 'spingere',
    'calcia': 'calciare'
}

it_pluralObjects = {
    'pesce': 'pesci',
    'topo': 'topi',
    'bottiglia': 'bottiglie',
    'giornale': 'giornali',
    'gesso': 'gessi',
    'scatola': 'scatole',
    'cappello': 'cappelli',
    'lampadina': 'lampadine',
    'tazza': 'tazze',
    'giocattolo': 'giocattoli',
    'scala': 'scale',
    'roccia': 'rocce',
    'collina': 'colline',
    'montagna': 'montagne',
    'tetto': 'tetti',
    'albero': 'alberi',
    'biscotto': 'biscotti',
    'banana': 'banane',
    'pera': 'pere',
    'pasto': 'pasti',
    'frutta': 'frutta',
    'cetriolo': 'cetrioli',
    'pizza': 'pizze',
    'libro': 'libri',
    'poema': 'poemi',
    'storia': 'storie',
    'capitolo': 'capitolo'
}

it_passiveSeed = {
'colpisce': { 'la': 'è colpita', 'una': 'è colpita', 'il': 'è colpito' , 'un': 'è colpito', 'uno': 'è colpito' },
'calcia': { 'la': 'è calciata', 'una': 'è calciata', 'il': 'è calciato' , 'un': 'è calciato', 'uno': 'è calciato' },
'tocca': { 'la': 'è toccata', 'una': 'è toccata', 'il': 'è toccato' , 'un': 'è toccato', 'uno': 'è toccato' },
'spinge': { 'la': 'è spinta', 'una': 'è spinta', 'il': 'è spinto' , 'un': 'è spinto', 'uno': 'è spinto' },
'beve': { 'la': 'è bevuta', 'una': 'è bevuta', 'il': 'è bevuto' , 'un': 'è bevuto', 'uno': 'è bevuto' },
'mangia': { 'la': 'è mangiata', 'una': 'è mangiata', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
'legge': { 'la': 'è letta', 'una': 'è letta', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' },
'colpiscono': { 'la': 'è colpita', 'una': 'è colpita', 'il': 'è colpito' , 'un': 'è colpito', 'uno': 'è colpito' },
'calciano': { 'la': 'è calciata', 'una': 'è calciata', 'il': 'è calciato' , 'un': 'è calciato', 'uno': 'è calciato' },
'toccano': { 'la': 'è toccata', 'una': 'è toccata', 'il': 'è toccato' , 'un': 'è toccato', 'uno': 'è toccato' },
'spingono': { 'la': 'è spinta', 'una': 'è spinta', 'il': 'è spinto' , 'un': 'è spinto', 'uno': 'è spinto' },
'bevono': { 'la': 'è bevuta', 'una': 'è bevuta', 'il': 'è bevuto' , 'un': 'è bevuto', 'uno': 'è bevuto' },
'mangiano': { 'la': 'è mangiata', 'una': 'è mangiata', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
'leggono': { 'la': 'è letta', 'una': 'è letta', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' }
}

verbDict_jp = {
    "eats": "食べる",
    "reads": "読む",
    "eat": "食べる",
    "read": "読む",
    "drinks": "飲む",
    "drink": "飲む",
    "hit": "打つ",
    "hits": "打つ",
    "kick": "蹴る",
    "kicks": "蹴る",
    "push": "押す",
    "pushes": "押す",
    "touch": "触る",
    "touches": "触る"
}

passiveDict_jp = {
    "食べる": "食べられる",
    "読む": "読まれる",
    "飲む": "飲まれる",
    "打つ": "打たれる",
    "蹴る": "蹴られる",
    "押す": "押される",
    "触る": "触られる"
}

particleDict_jp = {
    "wa": "は",
    "ga": "が",
    "o": "を",
    "to": "と",
    "ni": "に",
    "nai": "ない",
    "ta": "た"
}

nounDict_jp = {
    "lemonade": "レモネード",
    "espresso": "エスプレッソ",
    "milkshake": "ミルクセーキ",
    "smoothie": "スムージー",
    "beverage": "飲み物",
    "tonic": "トニック",
    "water": "水",
    "coffee": "コーヒー",
    "tea": "お茶",
    "juice": "ジュース",
    "pear": "梨",
    "author": "著者",
    "banana": "バナナ",
    "biscuit": "ビスケット",
    "book": "本",
    "lightbulb": "電球",
    "bottle": "ボトル",
    "box": "箱",
    "boy": "男の子",
    "bulb": "電球",
    "cap": "帽子",
    "cat": "猫",
    "chapter": "章",
    "chalk": "白亜",
    "cup": "コップ",
    "cucumber": "胡瓜",
    "dog": "犬",
    "fish": "魚",
    "fruit": "果物",
    "girl": "女の子",
    "hill": "丘",
    "man": "男",
    "meal": "食事",
    "mountain": "山",
    "mouse": "マウス",
    "newspaper": "新聞",
    "noodles": "麺",
    "poet": "詩人",
    "poem": "詩",
    "rock": "岩石",
    "roof": "屋根",
    "speaker": "スピーカー",
    "staircase": "階段",
    "story": "小説",
    "teacher": "先生",
    "toy": "玩具",
    "tree": "木",
    "woman": "女",
    "writer": "著者",
    "pizza": "ピザ",
    "pears": "梨",
    "authors": "著者",
    "bananas": "バナナ",
    "biscuits": "ビスケット",
    "books": "本",
    "bottles": "ボトル",
    "boxes": "箱",
    "boys": "男の子",
    "bulbs": "電球",
    "caps": "帽子",
    "cats": "猫",
    "chapters": "章",
    "chalks": "白亜",
    "cups": "コップ",
    "cucumbers": "胡瓜",
    "dogs": "犬",
    "fish": "魚",
    "fruits": "果物",
    "girls": "女の子",
    "hills": "丘",
    "men": "男性",
    "meals": "食事",
    "mountains": "山",
    "mouses": "マウス",
    "newspapers": "新聞",
    "noodles": "麺",
    "poets": "詩人",
    "poems": "詩",
    "rocks": "岩石",
    "roofs": "屋根",
    "speakers": "スピーカー",
    "staircases": "階段",
    "stories": "小説",
    "teachers": "先生",
    "toys": "玩具",
    "trees": "木",
    "women": "女性",
    "writers": "著者",
    "pizzas": "ピザ",
    "teacher": "教師",
    "boy": "少年",
    "man": "男",
    "men": "男性 ",
    "pole": "柱",
    "cat": "猫",
    "architect": "建築家",
    "author": "著者",
    "cats": "猫たち",
    "writer": "作家",
    "cooks": "料理人たち",
    "roof": "屋根",
    "women": "女性たち",
    "professor": "教授",
    "girls": "女の子たち",
    "lawyers": "弁護士たち",
    "professors": "教授たち",
    "orators": "演説家たち",
    "authors": "著者たち",
    "actresses": "女優たち",
    "orator": "演説家",
    "girl": "女の子",
    "dogs": "犬たち",
    "teachers": "教師たち",
    "dog": "犬",
    "poet": "詩人",
    "doctors": "医者たち",
    "tree": "木",
    "actress": "女優",
    "mountain": "山",
    "poets": "詩人たち",
    "writers": "作家たち",
    "doctor": "医者",
    "boys": "少年たち",
    "wall": "壁",
    "lawyer": "弁護士",
    "cook": "料理人",
    "architects": "建築家たち",
    "woman": "女性",
    "ladder": "梯子",
    "letter": "手紙",
    "magazine": "雑誌",
    "guide": "ガイド",
    "postcard": "ポストカード",
    "novella": "中編小説",
    "page": "ページ",
    "hat": "帽子",
    "mount": "マウント"
}

subordinateDict_jp = {
    'to think': '考える',
    'to say': "言う",
    'to state': '述べる',
    'to believe': '信じる',
    'to suggest': '提案する'
}

nameDict_jp = {
  "Sheela": "シーラ",
  "Leela": "リーラ",
  "Maria": "マリア",
  "Gomu": "ゴム",
  "John": "ジョン",
  "Tom": "トム",
  "Harry": "ハリー"
}


# Generate sentences

## English

In [4]:
seed = [{'verb': ['hits', 'kicks', 'touches', 'pushes'],
   'subject': ['dog', 'cat', 'man', 'teacher', 'boy'],
   'object': ['bottle', 'newspaper', 'box', 'lightbulb', 'cup']},

{'verb': ['drinks'],
'subject': ['dog', 'cat', 'man', 'teacher', 'boy'],
'object': ['juice', 'tea', 'coffee', 'water', 'tonic']},

{'verb': ['eats'],
'subject': ['dog', 'cat', 'man', 'teacher', 'boy'],
'object': ['banana', 'pear', 'fruit', 'pizza']},

{'verb': ['reads'],
'subject': ['poet', 'author', 'writer', 'orator', 'teacher', 'boy'],
'object': ['story', 'letter', 'magazine', 'guide', 'postcard', 'novella', 'page', 'poem']}]

seed_female = [{'verb': ['hits', 'kicks', 'touches', 'pushes'],
   'subject': ['woman', 'girl', 'actress', 'teacher', 'doctor', 'lawyer', 'architect', 'professor', 'writer', 'cook'],
   'object': ['fish', 'mouse', 'newspaper', 'chalk', 'hat', 'toy']},

{'verb': ['drinks'],
 'subject': ['woman', 'girl', 'actress', 'teacher', 'doctor', 'lawyer', 'architect', 'professor', 'writer', 'cook'],
 'object': ['beverage', 'lemonade', 'milkshake', 'smoothie', 'espresso']},

{'verb': ['eats'],
 'subject': ['woman', 'girl', 'actress', 'teacher', 'doctor', 'lawyer', 'architect', 'professor', 'writer', 'cook'],
 'object': ['biscuit', 'fish', 'meal', 'cucumber']},

{'verb': ['reads'],
 'subject': ['woman', 'girl', 'actress', 'teacher', 'doctor', 'lawyer', 'architect', 'professor', 'writer', 'cook'],
 'object': ['book', 'poem', 'chapter']}]

subordinateSeed = [ { 'verb' : ['thinks', 'says', 'states', 'believes', 'suggests'],
                     'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

#Genders of subjects and objects should be different.
it_seed = [ { 'verb' : ['colpisce', 'calcia', 'tocca', 'spinge'],  
             'subject': ['cane', 'gatto', 'uomo', 'insegnante', 'ragazzo'], 
             'object': ['bottiglia', 'giornale', 'scatola', 'lampadina', 'tazza']},

{ 'verb': ['beve'], 
 'subject': ['cane', 'gatto', 'uomo', 'insegnante', 'ragazzo'], 
 'object': ['succo', 'tè', 'caffè', 'acqua', 'tonico'] },

{ 'verb': ['mangia'], 
 'subject' : ['cane', 'gatto', 'uomo', 'insegnante', 'ragazzo'],
 'object': ['banana', 'pera', 'frutta', 'pizza' ]},

{'verb': ['legge'], 
 'subject' : ['poeta', 'autore', 'scrittore', 'oratorio', 'insegnante', 'ragazzo'], 
 'object': ['storia', 'lettera', 'rivista', 'guida', 'cartolina', 'novella', 'pagina', 'poesia']} ]

it_seed_female = [ { 'verb' : ['colpisce', 'calcia', 'tocca', 'spinge'],  
                    'subject': ['donna', 'ragazza', 'attrice', 'maestra', 'dottoressa', 'avvocata', 'architetta', 'professoressa', 'scrittrice', 'cuoca' ], 
                    'object': ['pesce', 'topo', 'giornale', 'gesso', 'cappello', 'giocattolo']},

{ 'verb': ['beve'], 
 'subject': ['donna', 'ragazza', 'attrice', 'maestra', 'dottoressa', 'avvocata', 'architetta', 'professoressa', 'scrittrice', 'cuoca' ], 
 'object': ['bevanda', 'limonata', 'frappè', 'frullato', 'espresso'] },

{ 'verb': ['mangia'], 
 'subject' : ['donna', 'ragazza', 'attrice', 'maestra', 'dottoressa', 'avvocata', 'architetta', 'professoressa', 'scrittrice', 'cuoca' ], 
 'object': ['biscotto', 'pesce', 'pasto', 'cetriolo']},

{'verb': ['legge'], 
 'subject' : ['donna', 'ragazza', 'attrice', 'maestra', 'dottoressa', 'avvocata', 'architetta', 'professoressa', 'scrittrice', 'cuoca' ], 
 'object': ['libro', 'poema', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['pensa', 'dice', 'afferma', 'crede', 'suggerisce'],  
                        'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

df = pd.DataFrame()

def make_df(seed, it_seed, it_genderDict):
    global df
    for oidx, obj in enumerate(seed):
        for sidx, subj in enumerate(obj['subject']):
            for obidx, ob in enumerate(obj['object']):
                for vidx, verb in enumerate(obj['verb']):
                    sdet = random.choice(['the', 'a'])
                    odet = random.choice(['the', 'a'])
                    pSubj = random.choice(subordinateSeed[0]['subject'])
                    pVerb = random.choice(subordinateSeed[0]['verb'])
    
                    if sdet == 'the':
                        it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][0] 
                    else: 
                        it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][1]
    
                    if odet == 'the':
                        it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][0] 
                    else: 
                        it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][1]
                    
                    it_subj = it_seed[oidx]['subject'][sidx]
                    it_ob = it_seed[oidx]['object'][obidx]
                    it_verb = it_seed[oidx]['verb'][vidx]
                    
                    it_pVerb = it_subordinateSeed[0]['verb'][subordinateSeed[0]['verb'].index(pVerb)]
    
                    temp_odet = it_odet
                    if it_sdet =='il':
                        ita_passive_from = 'dal'
                    if it_sdet =='la':
                        ita_passive_from = 'dalla'
                    elif it_sdet == "un" or it_sdet == "un'":
                        ita_passive_from = 'da un'
                        temp_odet = 'un'
                    elif it_sdet == "una":
                        ita_passive_from = 'da una'
                    elif it_sdet == "l'":
                        temp_odet = 'il'
                        ita_passive_from = "dall'"
                    if not "'" in ita_passive_from:
                        ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from} {it_subj}"
                    else:
                        ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from}{it_subj}"
                    # print(ita_passive_sentence)
                    if (it_sdet != "l'" and it_odet != "l'"):
                        wh = random.choice(['when', 'what', 'who', 'which', 'why'])
                        df = pd.concat([df, pd.DataFrame.from_dict([{
                            "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                            "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                            "ita-r-2-subordinate": f"{pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                            "ita-r-3-passive": ita_passive_sentence,
                            "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                            "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                            "ita-u-3-gender":f"{it_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                            "en": f"{sdet} {subj} {verb} {odet} {ob}",
                            "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                            "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                            "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                            "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                            "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?",
                            "en-u-4-wh": f"Did {sdet} {subj} {verb} {odet} {wh} {ob}?"
                        }])])
                    elif (it_sdet == "l'" and it_odet != "l'"):
                        wh = random.choice(['when', 'what', 'who', 'which', 'why'])
                        df = pd.concat([df, pd.DataFrame.from_dict([{
                            "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                            "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                            "ita-r-2-subordinate": f"{pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                            "ita-r-3-passive": ita_passive_sentence,
                            "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} {it_odet} no {it_ob}",
                            "ita-u-2-invert": " ".join(f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                            "ita-u-3-gender": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}", #using sdet instead of odet for subj, because it's a word starting w a vowel, and therefore the det is not gendered
                            "en": f"{sdet} {subj} {verb} {odet} {ob}",
                            "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                            "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                            "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                            "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                            "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?",
                            "en-u-4-wh": f"Did {sdet} {subj} {verb} {odet} {wh} {ob}?"
                        }])])
                    elif (it_sdet == "l'" and it_odet == "l'"):
                        wh = random.choice(['when', 'what', 'who', 'which', 'why'])
                        df = pd.concat([df, pd.DataFrame.from_dict([{
                            "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                            "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                            "ita-r-2-subordinate": f"{pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                            "ita-r-3-passive":ita_passive_sentence, 
                            "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} il no {it_ob}",
                            "ita-u-2-invert": " ".join( f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                            "ita-u-3-gender": f"{it_odet}{it_subj} {it_verb} {it_odet}{it_ob}", 
                            "en": f"{sdet} {subj} {verb} {odet} {ob}", 
                            "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}", 
                            "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}",
                            "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                            "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                            "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?",
                            "en-u-4-wh": f"Did {sdet} {subj} {verb} {odet} {wh} {ob}?"
                        }])])
                    else:
                        wh = random.choice(['when', 'what', 'who', 'which', 'why'])
                        df = pd.concat([df, pd.DataFrame.from_dict([{
                            "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                            "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                            "ita-r-2-subordinate": f"{pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                            "ita-r-3-passive":ita_passive_sentence, 
                            "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                            "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                            "ita-u-3-gender": f"il {it_subj} {it_verb} {it_odet}{it_ob}",
                            "en": f"{sdet} {subj} {verb} {odet} {ob}",
                            "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                            "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}", 
                            "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                            "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                            "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?",
                            "en-u-4-wh": f"Did {sdet} {subj} {verb} {odet} {wh} {ob}?"
                        }])])

make_df(seed, it_seed, it_genderDict)
make_df(seed_female, it_seed_female, it_genderDict)

In [5]:
pluralSeed = [{'verb': ['hit', 'kick', 'touch', 'push'],
   'subject': ['dogs', 'cats', 'men', 'teachers', 'boys'],
   'object': ['bottle', 'newspaper', 'box', 'lightbulb', 'cup']},

{'verb': ['drink'],
'subject': ['dogs', 'cats', 'men', 'teachers', 'boys'],
'object': ['juice', 'tea', 'coffee', 'water', 'tonic']},

{'verb': ['eat'],
'subject': ['dogs', 'cats', 'men', 'teachers', 'boys'],
'object': ['banana', 'pear', 'fruit', 'pizza']},

{'verb': ['read'],
'subject': ['poets', 'authors', 'writers', 'orators', 'teachers', 'boys'],
'object': ['story', 'letter', 'magazine', 'guide', 'postcard', 'novella', 'page', 'poem']}]

pluralSeed_female = [{'verb': ['hit', 'kick', 'touch', 'push'],
   'subject': ['women', 'girls', 'actresses', 'teachers', 'doctors', 'lawyers', 'architects', 'professors', 'writers', 'cooks'],
   'object': ['fish', 'mouse', 'newspaper', 'chalk', 'hat', 'toy']},

{'verb': ['drink'],
 'subject': ['women', 'girls', 'actresses', 'teachers', 'doctors', 'lawyers', 'architects', 'professors', 'writers', 'cooks'],
 'object': ['beverage', 'lemonade', 'milkshake', 'smoothie', 'espresso']},

{'verb': ['eat'],
 'subject': ['women', 'girls', 'actresses', 'teachers', 'doctors', 'lawyers', 'architects', 'professors', 'writers', 'cooks'],
 'object': ['biscuit', 'fish', 'meal', 'cucumber']},

{'verb': ['read'],
 'subject': ['women', 'girls', 'actresses', 'teachers', 'doctors', 'lawyers', 'architects', 'professors', 'writers', 'cooks'],
 'object': ['book', 'poem', 'chapter']}]

it_pluralSeed = [ { 'verb' : ['colpiscono', 'calciano', 'toccano', 'spingono'],  
             'subject': ['cani', 'gatti', 'uomini', 'insegnanti', 'ragazzi'], 
             'object': ['bottiglia', 'giornale', 'scatola', 'lampadina', 'tazza']},

{ 'verb': ['bevono'], 
 'subject': ['cani', 'gatti', 'uomini', 'insegnanti', 'ragazzi'], 
 'object': ['succo', 'tè', 'caffè', 'acqua', 'tonico'] },

{ 'verb': ['mangiano'], 
 'subject' : ['cani', 'gatti', 'uomini', 'insegnanti', 'ragazzi'],
 'object': ['banana', 'pera', 'frutta', 'pizza' ]},

{'verb': ['leggono'], 
 'subject' : ['poeti', 'autori', 'scrittori', 'oratori', 'insegnanti', 'ragazzi'], 
 'object': ['storia', 'lettera', 'rivista', 'guida', 'cartolina', 'novella', 'pagina', 'poesia']} ]

it_pluralSeed_female = [ { 'verb' : ['colpiscono', 'calciano', 'toccano', 'spingono'],  
                    'subject': ['donne', 'ragazze', 'attrici', 'maestre', 'dottoresse', 'avvocate', 'architette', 'professoresse', 'scrittrici', 'cuoche' ], 
                    'object': ['pesce', 'topo', 'giornale', 'gesso', 'cappello', 'giocattolo']},

{ 'verb': ['bevono'], 
 'subject': ['donne', 'ragazze', 'attrici', 'maestre', 'dottoresse', 'avvocate', 'architette', 'professoresse', 'scrittrici', 'cuoche' ], 
 'object': ['bevanda', 'limonata', 'frappè', 'frullato', 'espresso'] },

{ 'verb': ['mangiano'], 
 'subject' : ['donne', 'ragazze', 'attrici', 'maestre', 'dottoresse', 'avvocate', 'architette', 'professoresse', 'scrittrici', 'cuoche' ], 
 'object': ['biscotto', 'pesce', 'pasto', 'cetriolo']},

{'verb': ['leggono'], 
 'subject' : ['donne', 'ragazze', 'attrici', 'maestre', 'dottoresse', 'avvocate', 'architette', 'professoresse', 'scrittrici', 'cuoche' ], 
 'object': ['libro', 'poema', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['pensa', 'dice', 'afferma', 'crede', 'suggerisce'],
                        'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

def make_plural_df(pluralSeed, it_pluralSeed, it_genderDict):
    global df
    for oidx, obj in enumerate(pluralSeed):
        for sidx, subj in enumerate(obj['subject']):
            for obidx, ob in enumerate(obj['object']):
                for vidx, verb in enumerate(obj['verb']):
                    pSubj = random.choice(subordinateSeed[0]['subject'])
                    pVerb = random.choice(subordinateSeed[0]['verb'])
                    odet = random.choice(['the', 'a'])
                    
                    it_subj = it_pluralSeed[oidx]['subject'][sidx]
                    it_sdet = it_genderDict[it_subj][0]
                    it_ob = it_pluralSeed[oidx]['object'][obidx]
                    it_odet = it_genderDict[it_ob][0]
    
                    it_verb = it_pluralSeed[oidx]['verb'][vidx]
                    
                    it_pVerb = it_subordinateSeed[0]['verb'][subordinateSeed[0]['verb'].index(pVerb)]
    
                    if it_sdet == 'i':
                        ita_passive_from = 'dai'
                    if it_sdet == 'le':
                        ita_passive_from = 'dalle'
                    elif it_sdet == "gli":
                        ita_passive_from = "dagli'"

                    temp_odet = it_odet
                    if it_odet == "l'":
                        temp_odet = 'il'
                    else:
                        temp_odet = it_odet

                    if not "'" in ita_passive_from:
                        ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from} {it_subj}"
                    else:
                        ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from}{it_subj}"
    
                    wh = random.choice(['when', 'what', 'who', 'which', 'why'])
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}", 
                        "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}", 
                        "ita-r-2-subordinate": f"{pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-3-passive": ita_passive_sentence,
                        "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]), 
                        "ita-u-3-gender": f"{temp_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "en": f"the {subj} {verb} {odet} {ob}",
                        "en-r-1-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-2-passive": f"{odet} {ob} is {passiveSeed[verb]} by the {subj}",
                        "en-u-1-negation": f"the {subj} {verb} {odet} doesn't {ob}",
                        "en-u-2-inversion": " ".join(f"the {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                        "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?",
                        "en-u-4-wh": f"Did the {subj} {verb} {odet} {wh} {ob}?"
                    }])])
                    if (f"{odet} {ob} is {passiveSeed[verb]} by the {subj}" == None):
                        print(f"{odet} {ob} is {passiveSeed[verb]} by the {subj}", odet, ob, passiveSeed[verb], subj)

make_plural_df(pluralSeed, it_pluralSeed, it_genderDict)
make_plural_df(pluralSeed_female, it_pluralSeed_female, it_genderDict)

df.reset_index()

,index,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,en-u-1-negation,en-u-2-inversion,en-u-3-qsubordinate,en-u-4-wh
0,0,il cane colpisce la bottiglia,colpisce la bottiglia,Tom afferma che il cane colpisce la bottiglia,la bottiglia è colpita dal cane,il cane colpisce la no bottiglia,bottiglia la colpisce cane il,la cane colpisce la bottiglia,the dog hits the bottle,Tom states that the dog hits the bottle,the bottle is hit by the dog,the dog hits the doesn't bottle,bottle the hits dog the,Tom states that does the dog hits the bottle?,Did the dog hits the what bottle?
1,0,il cane calcia la bottiglia,calcia la bottiglia,Tom pensa che il cane calcia la bottiglia,la bottiglia è calciata dal cane,il cane calcia la no bottiglia,bottiglia la calcia cane il,la cane calcia la bottiglia,the dog kicks the bottle,Tom thinks that the dog kicks the bottle,the bottle is kicked by the dog,the dog kicks the doesn't bottle,bottle the kicks dog the,Tom thinks that does the dog kicks the bottle?,Did the dog kicks the why bottle?
2,0,il cane tocca una bottiglia,tocca una bottiglia,Sheela pensa che il cane tocca una bottiglia,una bottiglia è toccata dal cane,il cane tocca una no bottiglia,bottiglia una tocca cane il,una cane tocca una bottiglia,the dog touches a bottle,Sheela thinks that the dog touches the bottle,a bottle is touched by the dog,the dog touches a doesn't bottle,bottle a touches dog the,Sheela thinks that does the dog touches the bo...,Did the dog touches a when bottle?
3,0,il cane spinge la bottiglia,spinge la bottiglia,John suggerisce che il cane spinge la bottiglia,la bottiglia è spinta dal cane,il cane spinge la no bottiglia,bottiglia la spinge cane il,la cane spinge la bottiglia,the dog pushes the bottle,John suggests that the dog pushes the bottle,the bottle is pushed by the dog,the dog pushes the doesn't bottle,bottle the pushes dog the,John suggests that does the dog pushes the bot...,Did the dog pushes the when bottle?
4,0,il cane colpisce un giornale,colpisce un giornale,Leela crede che il cane colpisce un giornale,un giornale è colpito dal cane,il cane colpisce un no giornale,giornale un colpisce cane il,un cane colpisce un giornale,the dog hits a newspaper,Leela believes that the dog hits the newspaper,a newspaper is hit by the dog,the dog hits a doesn't newspaper,newspaper a hits dog the,Leela believes that does the dog hits the news...,Did the dog hits a why newspaper?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,0,le scrittrici leggono il poema,leggono il poema,Tom pensa che le scrittrici leggono il poema,il poema è letto dalle scrittrici,le scrittrici leggono il no poema,poema il leggono scrittrici le,il scrittrici leggono il poema,the writers read the poem,Tom thinks that the writers read the poem,the poem is read by the writers,the writers read the doesn't poem,poem the read writers the,Tom thinks that does the writers read the poem?,Did the writers read the when poem?
1102,0,le scrittrici leggono il capitolo,leggono il capitolo,Maria crede che le scrittrici leggono il capitolo,il capitolo è letto dalle scrittrici,le scrittrici leggono il no capitolo,capitolo il leggono scrittrici le,il scrittrici leggono il capitolo,the writers read a chapter,Maria believes that the writers read the chapter,a chapter is read by the writers,the writers read a doesn't chapter,chapter a read writers the,Maria believes that does the writers read the ...,Did the writers read a which chapter?
1103,0,le cuoche leggono il libro,leggono il libro,Gomu suggerisce che le cuoche leggono il libro,il libro è letto dalle cuoche,le cuoche leggono il no libro,libro il leggono cuoche le,il cuoche leggono il libro,the cooks read the book,Gomu suggests that the cooks read the book,the book is read by the cooks,the cooks read the doesn't book,book the read cooks the,Gomu suggests that does the cooks read the book?,Did the cooks read t

## Italian sentence

In [6]:
def get_gender(key):
    if 'la' in it_genderDict[en_it[key]]:
        return 'kar'
    elif 'il' in it_genderDict[en_it[key]] or "l'" in it_genderDict[en_it[key]]:
        return 'kon'
    elif 'le' in it_genderDict[en_it[key]]:
        return 'kars'
    else:
        return 'kons'
df['it'] = df.progress_apply(lambda row: " ".join([get_gender(row['en'].split(" ")[1])] + 
                                                   row['en'].split(" ")[1:3] + 
                                                   [get_gender(row['en'].split(" ")[4])] + 
                                                   [row['en'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 112691.91it/s]


## IT Real grammar 1 (Null Subject parameter)

In [7]:
df['it-r-1-null_subject'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[2:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 155942.07it/s]


## IT Real Grammar 2 (Passive construction)

In [8]:
df['it-r-2-passive'] =  df.progress_apply(lambda row: " ".join([get_gender(row['en-r-2-passive'].split(" ")[1])] + row['en-r-2-passive'].split(" ")[1:-2] + [get_gender(row['en-r-2-passive'].split(" ")[-1])] + [row['en-r-2-passive'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 125267.58it/s]


## IT Real Grammar 3 (Subordinate construction)

In [9]:
df['it-r-3-subordinate'] =  df.progress_apply(lambda row: " ".join(row['en-r-1-subordinate'].split(" ")[0:3] + row['it'].split(" ")), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 189346.46it/s]


In [10]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,en-u-1-negation,en-u-2-inversion,en-u-3-qsubordinate,en-u-4-wh,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate
0,il cane colpisce la bottiglia,colpisce la bottiglia,Tom afferma che il cane colpisce la bottiglia,la bottiglia è colpita dal cane,il cane colpisce la no bottiglia,bottiglia la colpisce cane il,la cane colpisce la bottiglia,the dog hits the bottle,Tom states that the dog hits the bottle,the bottle is hit by the dog,the dog hits the doesn't bottle,bottle the hits dog the,Tom states that does the dog hits the bottle?,Did the dog hits the what bottle?,kon dog hits kar bottle,hits kar bottle,kar bottle is hit by kon dog,Tom states that kon dog hits kar bottle
0,il cane calcia la bottiglia,calcia la bottiglia,Tom pensa che il cane calcia la bottiglia,la bottiglia è calciata dal cane,il cane calcia la no bottiglia,bottiglia la calcia cane il,la cane calcia la bottiglia,the dog kicks the bottle,Tom thinks that the dog kicks the bottle,the bottle is kicked by the dog,the dog kicks the doesn't bottle,bottle the kicks dog the,Tom thinks that does the dog kicks the bottle?,Did the dog kicks the why bottle?,kon dog kicks kar bottle,kicks kar bottle,kar bottle is kicked by kon dog,Tom thinks that kon dog kicks kar bottle
0,il cane tocca una bottiglia,tocca una bottiglia,Sheela pensa che il cane tocca una bottiglia,una bottiglia è toccata dal cane,il cane tocca una no bottiglia,bottiglia una tocca cane il,una cane tocca una bottiglia,the dog touches a bottle,Sheela thinks that the dog touches the bottle,a bottle is touched by the dog,the dog touches a doesn't bottle,bottle a touches dog the,Sheela thinks that does the dog touches the bo...,Did the dog touches a when bottle?,kon dog touches kar bottle,touches kar bottle,kar bottle is touched by kon dog,Sheela thinks that kon dog touches kar bottle
0,il cane spinge la bottiglia,spinge la bottiglia,John suggerisce che il cane spinge la bottiglia,la bottiglia è spinta dal cane,il cane spinge la no bottiglia,bottiglia la spinge cane il,la cane spinge la bottiglia,the dog pushes the bottle,John suggests that the dog pushes the bottle,the bottle is pushed by the dog,the dog pushes the doesn't bottle,bottle the pushes dog the,John suggests that does the dog pushes the bot...,Did the dog pushes the when bottle?,kon dog pushes kar bottle,pushes kar bottle,kar bottle is pushed by kon dog,John suggests that kon dog pushes kar bottle
0,il cane colpisce un giornale,colpisce un giornale,Leela crede che il cane colpisce un giornale,un giornale è colpito dal cane,il cane colpisce un no giornale,giornale un colpisce cane il,un cane colpisce un giornale,the dog hits a newspaper,Leela believes that the dog hits the newspaper,a newspaper is hit by the dog,the dog hits a doesn't newspaper,newspaper a hits dog the,Leela believes that does the dog hits the news...,Did the dog hits a why newspaper?,kon dog hits kon newspaper,hits kon newspaper,kon newspaper is hit by kon dog,Leela believes that kon dog hits kon newspaper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le scrittrici leggono il poema,leggono il poema,Tom pensa che le scrittrici leggono il poema,il poema è letto dalle scrittrici,le scrittrici leggono il no poema,poema il leggono scrittrici le,il scrittrici leggono il poema,the writers read the poem,Tom thinks that the writers read the poem,the poem is read by the writers,the writers read the doesn't poem,poem the read writers the,Tom thinks that does the writers read the poem?,Did the writers read the when poem?,kars writers read kon poem,read kon poem,kon poem is read by kars writers,Tom thinks that kars writers read kon poem
0,le scrittrici leggono il capitolo,leggono il capitolo,Maria crede che le scrittrici leggono il capitolo,il capitolo è letto dalle scrittrici,le scrittrici leggono il no capitolo,capito

## IT Unreal Grammar 1: Add a negation after the 3rd word in the nullified subject sentence

In [11]:
df['it-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:4] + [ "no" ] + row['en'].split(" ")[4:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 181360.82it/s]


## IT Unreal Grammar 2: Invert italian sentence

In [12]:
df['it-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 161052.39it/s]


## IT Unreal Grammar 3: Same gender for subject and object

In [13]:
df['it-u-3-gender'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:3] + [row['it'].split(" ")[0]] + [row['it'].split(" ")[-1]]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 156400.39it/s]


## JP real grammar 1 (Wa after subj, o after obj, verb)

In [14]:
df['jap-r-1-sov'] = df.progress_apply(lambda row: f'{nounDict_jp[row["jp-r-1-sov"].split(" ")[1]]} {particleDict_jp["wa"]} {nounDict_jp[row["jp-r-1-sov"].split(" ")[4]]} {particleDict_jp["o"]} {verbDict_jp[row["jp-r-1-sov"].split(" ")[-1]]}' , axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 150878.25it/s]


## JP real grammar 2 (Passive construction)

In [15]:
df['jp-r-2-passive'] = df.progress_apply(lambda row: " ".join(row["en"].split(" ")[3:5]) + " wa " + " ".join(row["en"].split(" ")[:2]) + " ni " + infinitive[row["en"].split(" ")[2]] + " reru", axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 151018.75it/s]


In [16]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,...,en-u-3-qsubordinate,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive
0,il cane porta il pesce,porta il pesce,Leela dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog carries the fish,Tom notices that the dog carries the fish,the fish is carried by the dog,...,Tom notices that does the dog carries the fish?,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Tom notices that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,the dog wa the fish o carries,the fish wa the dog ni to carry reru
0,il cane tiene il pesce,tiene il pesce,Sheela afferma che il cane tiene il pesce,il pesce è tenuto dal cane,il cane tiene il no pesce,pesce il tiene cane il,il cane tiene il pesce,the dog holds the fish,Tom claims that the dog holds the fish,the fish is held by the dog,...,Tom claims that does the dog holds the fish?,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,Tom claims that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,the dog wa the fish o holds,the fish wa the dog ni to hold reru
0,un cane prende il pesce,prende il pesce,Leela dice che un cane prende il pesce,il pesce è preso dal cane,un cane prende il no pesce,pesce il prende cane un,il cane prende il pesce,a dog takes the fish,Sheela sees that the dog takes the fish,the fish is taken by a dog,...,Sheela sees that does the dog takes the fish?,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Sheela sees that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,a dog wa the fish o takes,the fish wa a dog ni to take reru
0,il cane porta il pesce,porta il pesce,Gomu dice che il cane porta il pesce,il pesce è portato dal cane,il cane porta il no pesce,pesce il porta cane il,il cane porta il pesce,the dog brings the fish,Tom claims that the dog brings the fish,the fish is brought by the dog,...,Tom claims that does the dog brings the fish?,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,Tom claims that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa the fish o brings,the fish wa the dog ni to bring reru
0,un cane porta un topo,porta un topo,Harry dice che un cane porta un topo,un topo è portato da un cane,un cane porta un no topo,topo un porta cane un,un cane porta un topo,a dog carries a mouse,Harry sees that the dog carries the mouse,a mouse is carried by a dog,...,Harry sees that does the dog carries the mouse?,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Harry sees that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa a mouse o carries,a mouse wa a dog ni to carry reru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono il capitolo,leggono il capitolo,Maria sa che le ragazze leggono il capitolo,il capitolo è letto dalle ragazze,le ragazze leggono il no capitolo,capitolo il leggono ragazze le,il ragazze leggono il capitolo,the girls read a chapter,Tom says that the girls read the chapter,a chapter is read by the girls,...,Tom says that does the girls read the chapter?,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Tom says that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa a chapter o read,a chapter wa the girls ni to read reru
0,i ragazzi leggono il libro,leggono il libro,Maria osserva che i ragazzi leggono il libro,il libro è l

## JP real grammar 3 (Subordinate construction)

In [17]:
df['jp-r-3-subordinate'] = df.progress_apply(lambda row: " ".join([row["en-r-1-subordinate"].split(" ")[0]] + ["wa"] + row["en-r-1-subordinate"].split(" ")[3:5] + ["ga"] + row["en-r-1-subordinate"].split(" ")[-2:]  + ["o"] + [row["en-r-1-subordinate"].split(" ")[5]] + ["to"] + [row["en-r-1-subordinate"].split(" ")[1][:-1]]), axis=1)

df['jap-r-3-subordinate'] = df.progress_apply(lambda row: f'{nameDict_jp[row["jp-r-3-subordinate"].split(" ")[0]]} {particleDict_jp["wa"]} {row["jap-r-1-sov"].replace(particleDict_jp["wa"],particleDict_jp["ga"])} {particleDict_jp["to"]} {subordinateDict_jp[" ".join(row["jp-r-3-subordinate"].split(" ")[-2:])]}' , axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 103977.25it/s]


## JP - Unreal grammar 1:Add a negation at the end of the object in the real-jp-1 sentence

In [18]:
df['jp-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:3]) + " no " + " ".join(row['jp-r-1-sov'].split(" ")[3:]), axis=1)

df['jap-u-1-negation'] = df.progress_apply(lambda row: f'{" ".join(row["jap-r-1-sov"].split(" ")[:3])} {particleDict_jp["nai"]} {" ".join(row["jap-r-1-sov"].split(" ")[3:])}', axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 177875.12it/s]


## JP - Unreal grammar 2: Invert jp-real-1 sentence

In [19]:
df['jp-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[::-1]), axis=1)

df['jap-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['jap-r-1-sov'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 245726.90it/s]


In [20]:
df

ita ita-r-1-null_subject  \
0           il cane porta il pesce       porta il pesce   
0           il cane tiene il pesce       tiene il pesce   
0          il cane prende il pesce      prende il pesce   
0           un cane porta il pesce       porta il pesce   
0            un cane porta un topo        porta un topo   
..                             ...                  ...   
0   le ragazze leggono il capitolo  leggono il capitolo   
0       i ragazzi leggono il libro     leggono il libro   
0       i ragazzi leggono il poema     leggono il poema   
0      i ragazzi leggono la storia    leggono la storia   
0    i ragazzi leggono il capitolo  leggono il capitolo   

                              ita-r-2-subordinate  \
0           Leela dice che il cane porta il pesce   
0          Sheela vede che il cane tiene il pesce   
0      Sheela afferma che il cane prende il pesce   
0           Maria vede che un cane porta il pesce   
0         Leela osserva che un cane porta un topo   
..                                            ...   
0   Harry dice che le ragazze leggono il capitolo   
0    Harry afferma che i ragazzi leggono il libro   
0        John dice che i ragazzi leggono il poema   
0      Leela vede che i ragazzi leggono la storia   
0        Tom sa che i ragazzi leggono il capitolo   

                      ita-r-3-passive                   ita-u-1-negation  \
0         il pesce è portato dal cane          il cane porta il no pesce   
0          il pesce è tenuto dal cane          il cane tiene il no pesce   
0           il pesce è preso dal cane         il cane prende il no pesce   
0         il pesce è portato dal cane          un cane porta il no pesce   
0        un topo è portato da un cane           un cane porta un no topo   
..                                ...                                ...   
0   il capitolo è letto dalle ragazze  le ragazze leggono il no capitolo   
0        il libro è letto dai ragazzi      i ragazzi leggono il no libro   
0        il poema è letto dai ragazzi      i ragazzi leggono il no poema   
0       la storia è letto dai ragazzi     i ragazzi leggono la no storia   
0     il capitolo è letto dai ragazzi   i ragazzi leggono il no capitolo   

                    ita-u-2-invert                  ita-u-3-gender  \
0           pesce il porta cane il          il cane porta il pesce   
0           pesce il tiene cane il          il cane tiene il pesce   
0          pesce il prende cane il         il cane prende il pesce   
0           pesce il porta cane un          il cane porta il pesce   
0            topo un porta cane un           un cane porta un topo   
..                             ...                             ...   
0   capitolo il leggono ragazze le  il ragazze leggono il capitolo   
0       libro il leggono ragazzi i     il ragazzi leggono il libro   
0       poema il leggono ragazzi i     il ragazzi leggono il poema   
0      storia la leggono ragazzi i    la ragazzi leggono la storia   
0    capitolo il leggono ragazzi i  il ragazzi leggono il capitolo   

                            en                           en-r-1-subordinate  \
0     the dog carries the fish    Tom notices that the dog carries the fish   
0       the dog holds the fish       John knows that the dog holds the fish   
0       the dog takes the fish     John affirms that the dog takes the fish   
0        a dog brings the fish    Gomu affirms that the dog brings the fish   
0        a dog carries a mouse    Maria says that the dog carries the mouse   
..                         ...                                          ...   
0   the girls read the chapter  Tom affirms that the girls read the chapter   
0       the boys read the book        Gomu sees that the boys read the book   
0       the boys read the poem       Leela sees that the boys read the poem   
0      the boys read the story      Maria says that the boys read the story   
0      the boys read a chapter   Harry knows that the boys rea

## JP - Unreal grammar add a after o + past tense

In [21]:
df['jp-u-3-past-tense'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:-2] + [' o-ta '] + [infinitive[row['jp-r-1-sov'].split(" ")[-1]]]), axis=1)
df['jap-u-3-past-tense'] = df.progress_apply(lambda row: f'{" ".join(row['jap-r-1-sov'].split(" ")[:-2])} {particleDict_jp['o']}-{particleDict_jp['ta']} {row['jap-r-1-sov'].split(" ")[-1]}', axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 181573.54it/s]


In [177]:
df

,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,en,en-r-1-subordinate,en-r-2-passive,...,jp-r-2-passive,jap-r-2-passive,jp-r-3-subordinate,jap-r-3-subordinate,jp-u-1-negation,jap-u-1-negation,jp-u-2-invert,jap-u-2-invert,jp-u-3-past-tense,jap-u-3-past-tense
0,il cane colpisce la bottiglia,colpisce la bottiglia,Tom afferma che il cane colpisce la bottiglia,la bottiglia è colpita dal cane,il cane colpisce la no bottiglia,bottiglia la colpisce cane il,la cane colpisce la bottiglia,the dog hits the bottle,Tom states that the dog hits the bottle,the bottle is hit by the dog,...,the bottle wa the dog ni to hit reru,ボトル は 犬 に 打たれる,Tom wa the dog ga the bottle o hits to state,トム は 犬 が ボトル を 打つ と 述べる,the dog wa no the bottle o hits,犬 は ボトル ない を 打つ,hits o bottle the wa dog the,打つ を ボトル は 犬,the dog wa the bottle o-ta to hit,犬 は ボトル を-た 打つ
0,il cane calcia la bottiglia,calcia la bottiglia,Tom pensa che il cane calcia la bottiglia,la bottiglia è calciata dal cane,il cane calcia la no bottiglia,bottiglia la calcia cane il,la cane calcia la bottiglia,the dog kicks the bottle,Tom thinks that the dog kicks the bottle,the bottle is kicked by the dog,...,the bottle wa the dog ni to kick reru,ボトル は 犬 に 蹴られる,Tom wa the dog ga the bottle o kicks to think,トム は 犬 が ボトル を 蹴る と 考える,the dog wa no the bottle o kicks,犬 は ボトル ない を 蹴る,kicks o bottle the wa dog the,蹴る を ボトル は 犬,the dog wa the bottle o-ta to kick,犬 は ボトル を-た 蹴る
0,il cane tocca una bottiglia,tocca una bottiglia,Sheela pensa che il cane tocca una bottiglia,una bottiglia è toccata dal cane,il cane tocca una no bottiglia,bottiglia una tocca cane il,una cane tocca una bottiglia,the dog touches a bottle,Sheela thinks that the dog touches the bottle,a bottle is touched by the dog,...,a bottle wa the dog ni to touch reru,ボトル は 犬 に 触られる,Sheela wa the dog ga the bottle o touches to t...,シーラ は 犬 が ボトル を 触る と 考える,the dog wa no a bottle o touches,犬 は ボトル ない を 触る,touches o bottle a wa dog the,触る を ボトル は 犬,the dog wa a bottle o-ta to touch,犬 は ボトル を-た 触る
0,il cane spinge la bottiglia,spinge la bottiglia,John suggerisce che il cane spinge la bottiglia,la bottiglia è spinta dal cane,il cane spinge la no bottiglia,bottiglia la spinge cane il,la cane spinge la bottiglia,the dog pushes the bottle,John suggests that the dog pushes the bottle,the bottle is pushed by the dog,...,the bottle wa the dog ni to push reru,ボトル は 犬 に 押される,John wa the dog ga the bottle o pushes to suggest,ジョン は 犬 が ボトル を 押す と 提案する,the dog wa no the bottle o pushes,犬 は ボトル ない を 押す,pushes o bottle the wa dog the,押す を ボトル は 犬,the dog wa the bottle o-ta to push,犬 は ボトル を-た 押す
0,il cane colpisce un giornale,colpisce un giornale,Leela crede che il cane colpisce un giornale,un giornale è colpito dal cane,il cane colpisce un no giornale,giornale un colpisce cane il,un cane colpisce un giornale,the dog hits a newspaper,Leela believes that the dog hits the newspaper,a newspaper is hit by the dog,...,a newspaper wa the dog ni to hit reru,新聞 は 犬 に 打たれる,Leela wa the dog ga the newspaper o hits to be...,リーラ は 犬 が 新聞 を 打つ と 信じる,the dog wa no a newspaper o hits,犬 は 新聞 ない を 打つ,hits o newspaper a wa dog the,打つ を 新聞 は 犬,the dog wa a newspaper o-ta to hit,犬 は 新聞 を-た 打つ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le scrittrici leggono il poema,leggono il poema,Tom pensa che le scrittrici leggono il poema,il poema è letto dalle scrittrici,le scrittrici leggono il no poema,poema il leggono scrittrici le,il scrittrici leggono il poema,the writers read the poem,Tom thinks that the writers read the poem,the poem is read by the writers,...,the poem wa the writers ni to read reru,詩 は 作家たち に 読まれる,Tom wa the writers ga the poem o read to think,トム は 作家たち が 詩 を 読む と 考える,the writers wa no the poem o read,作家たち は 詩 ない を 読む,read o poem the wa writers the,読む を 詩 は 作家たち,the writers wa the poem o-ta to read,作家たち は 詩 を-た 読む
0,le scrittrici leggono il capitolo,leggono il capi

In [22]:
df

ita ita-r-1-null_subject  \
0           il cane porta il pesce       porta il pesce   
0           il cane tiene il pesce       tiene il pesce   
0          il cane prende il pesce      prende il pesce   
0           un cane porta il pesce       porta il pesce   
0            un cane porta un topo        porta un topo   
..                             ...                  ...   
0   le ragazze leggono il capitolo  leggono il capitolo   
0       i ragazzi leggono il libro     leggono il libro   
0       i ragazzi leggono il poema     leggono il poema   
0      i ragazzi leggono la storia    leggono la storia   
0    i ragazzi leggono il capitolo  leggono il capitolo   

                              ita-r-2-subordinate  \
0           Leela dice che il cane porta il pesce   
0          Sheela vede che il cane tiene il pesce   
0      Sheela afferma che il cane prende il pesce   
0           Maria vede che un cane porta il pesce   
0         Leela osserva che un cane porta un topo   
..                                            ...   
0   Harry dice che le ragazze leggono il capitolo   
0    Harry afferma che i ragazzi leggono il libro   
0        John dice che i ragazzi leggono il poema   
0      Leela vede che i ragazzi leggono la storia   
0        Tom sa che i ragazzi leggono il capitolo   

                      ita-r-3-passive                   ita-u-1-negation  \
0         il pesce è portato dal cane          il cane porta il no pesce   
0          il pesce è tenuto dal cane          il cane tiene il no pesce   
0           il pesce è preso dal cane         il cane prende il no pesce   
0         il pesce è portato dal cane          un cane porta il no pesce   
0        un topo è portato da un cane           un cane porta un no topo   
..                                ...                                ...   
0   il capitolo è letto dalle ragazze  le ragazze leggono il no capitolo   
0        il libro è letto dai ragazzi      i ragazzi leggono il no libro   
0        il poema è letto dai ragazzi      i ragazzi leggono il no poema   
0       la storia è letto dai ragazzi     i ragazzi leggono la no storia   
0     il capitolo è letto dai ragazzi   i ragazzi leggono il no capitolo   

                    ita-u-2-invert                  ita-u-3-gender  \
0           pesce il porta cane il          il cane porta il pesce   
0           pesce il tiene cane il          il cane tiene il pesce   
0          pesce il prende cane il         il cane prende il pesce   
0           pesce il porta cane un          il cane porta il pesce   
0            topo un porta cane un           un cane porta un topo   
..                             ...                             ...   
0   capitolo il leggono ragazze le  il ragazze leggono il capitolo   
0       libro il leggono ragazzi i     il ragazzi leggono il libro   
0       poema il leggono ragazzi i     il ragazzi leggono il poema   
0      storia la leggono ragazzi i    la ragazzi leggono la storia   
0    capitolo il leggono ragazzi i  il ragazzi leggono il capitolo   

                            en                           en-r-1-subordinate  \
0     the dog carries the fish    Tom notices that the dog carries the fish   
0       the dog holds the fish       John knows that the dog holds the fish   
0       the dog takes the fish     John affirms that the dog takes the fish   
0        a dog brings the fish    Gomu affirms that the dog brings the fish   
0        a dog carries a mouse    Maria says that the dog carries the mouse   
..                         ...                                          ...   
0   the girls read the chapter  Tom affirms that the girls read the chapter   
0       the boys read the book        Gomu sees that the boys read the book   
0       the boys read the poem       Leela sees that the boys read the poem   
0      the boys read the story      Maria says that the boys read the story   
0      the boys read a chapter   Harry knows that the boys rea

# Non-grammatical sentences

In [23]:
import re
def swap_words(sentence, col):
    # print(col, sentence)
    

    swapProhibited = {
        'en': [ [0,3], [1,4] ],
        'en-r-1-subordinate': [ [0,4], [0,-1], [4,-1], [3,-2], [1,5] ],
        'en-r-2-passive': [ [0,-2], [1,-1] ],
        'en-u-1-negation': [ [0,3], [1,5] ],
        'en-u-2-inversion': [ [0,-2], [1,-1] ],
        'en-u-3-qsubordinate': [ [4,7], [5,8], [1,6], [0,4], [0, 7] ],
        'en-u-4-wh': [ [1,4], [2,-1] ],
        'it': [ [0,3], [1,4] ],
        'it-r-1-null_subject': [ ],
        'it-r-2-passive': [ [0,5], [1,6] ],
        'it-r-3-subordinate': [ [0,5], [1,6] ],
        'it-u-1-negation': [ [0,3], [1,5] ],
        'it-u-2-invert': [ [0,3], [1,4] ],
        'it-u-3-gender': [ [0,3], [1,4] ],
        'ita': [ [1,-1], [0,-2] ],
        'ita-r-1-null_subject': [ ],
        'ita-r-2-subordinate': [ [0,4], [0,-1], [4,-1], [3,-2], [1,-3] ],
        'ita-r-3-passive': [ [1,-1] ],
        'ita-u-1-negation': [ [0,3], [1,-1] ],
        'ita-u-2-invert': [ [0,-2], [1,-1] ],
        'ita-u-3-gender': [ [0,-2], [1,-1] ],
        'jp-r-1-sov': [ [0,3], [1,4] ],
        'jp-r-2-passive': [ [0,3], [1,4] ],
        'jp-r-3-subordinate': [ [0,3], [0,6], [3,6] ],
        'jp-u-1-negation': [ [0,4], [1,5] ],
        'jp-u-2-invert': [ [3,6], [2,5] ],
        'jp-u-3-past-tense': [ [0,3], [1,4] ],
        'jap-r-1-sov': [ [0,2] ],
        'jap-r-2-passive': [ [0,2]],
        'jap-r-3-subordinate': [ [0,2], [0,4], [2,4], [6,-1] ],
        'jap-u-1-negation': [ [0,2] ],
        'jap-u-2-invert': [ [2,-1] ],
        'jap-u-3-past-tense': [ [0,2] ]
    }
    # if (col == 'jap-r-1-sov'):
    #     print(sentence, len(sentence), len(sentence.split(" ")))
    if col == 'en-u-1-negation':
        sentence = re.split(" ", sentence)
    else:
        sentence = re.split("'| ", sentence)
        
    # if (col == 'jap-r-1-sov'):
    #     print(sentence, len(sentence))
    numWords = len(sentence)
    toSwap = random.sample(range(0, numWords), 2)
    toSwapWords = set([sentence[toSwap[0]],sentence[toSwap[1]]])
    prohibitedWords = [set([sentence[pos[0]],sentence[pos[1]]]) for pos in swapProhibited[col]]
    while toSwapWords in prohibitedWords:
        toSwap = random.sample(range(0, numWords), 2)
        toSwapWords = set([sentence[toSwap[0]],sentence[toSwap[1]]])
    swap1 = sentence[toSwap[0]]
    swap2 = sentence[toSwap[1]]
    sentence[toSwap[0]] = swap2
    sentence[toSwap[1]] = swap1
    sentence = " ".join(sentence)
    if '?' in sentence and sentence.index('?') != len(sentence) - 1:
        sentence = sentence.replace('?', '')
        sentence = sentence +'?'
    return sentence

for col in list(df.columns):
    print(' Now processing.... ', col)
    df[f'ng-{col}'] = df.progress_apply(lambda row: swap_words(row[col], col), axis=1)

 Now processing....  ita


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 229964.54it/s]


 Now processing....  ita-r-1-null_subject


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 216308.34it/s]


 Now processing....  ita-r-2-subordinate


...

 Now processing....  jp-u-2-invert


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 204994.27it/s]


 Now processing....  jp-u-3-past-tense


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 202159.13it/s]


  0%|          | 0/812 [00:00<?, ?it/s]

In [24]:
gCols = [col for col in df.columns if not 'ng' in col]
ngCols = [col for col in df.columns if 'ng' in col]
df[ngCols]

,ng-ita,ng-ita-r-1-null_subject,ng-ita-r-2-subordinate,ng-ita-r-3-passive,ng-ita-u-1-negation,ng-ita-u-2-invert,ng-ita-u-3-gender,ng-en,ng-en-r-1-subordinate,ng-en-r-2-passive,...,ng-jp-r-1-sov,ng-jap-r-1-sov,ng-jp-r-2-passive,ng-jap-r-2-passive,ng-jp-r-3-subordinate,ng-jp-u-1-negation,ng-jap-u-1-negation,ng-jp-u-2-invert,ng-jap-u-2-invert,ng-jp-u-3-past-tense
0,il cane porta pesce il,porta pesce il,Leela dice che il cane porta pesce il,il pesce è portato cane dal,il cane porta il pesce no,pesce il porta il cane,il cane porta pesce il,the dog carries fish the,Tom notices that the dog carries fish the,the fish is carried by dog the,...,the dog wa the fish carries o,犬 は 魚 運ぶ を,the fish wa the dog ni to reru carry,魚 は 犬 運ばれる に,Tom wa the dog ga the fish o carries notices to,the dog wa no the fish carries o,犬 は 魚 ない 運ぶ を,carries o fish the wa the dog,運ぶ を 魚 犬 は,the dog wa the fish o-ta carry to
0,il cane tiene pesce il,tiene pesce il,Sheela vede che il cane tiene pesce il,il pesce è tenuto cane dal,il cane tiene il pesce no,pesce il tiene il cane,il cane tiene pesce il,the dog holds fish the,John knows that the dog holds fish the,the fish is held by dog the,...,the dog wa the fish holds o,犬 は 魚 持つ を,the fish wa the dog ni to reru hold,魚 は 犬 持たれる に,John wa the dog ga the fish o holds knows to,the dog wa no the fish holds o,犬 は 魚 ない 持つ を,holds o fish the wa the dog,持つ を 魚 犬 は,the dog wa the fish o-ta hold to
0,il cane prende pesce il,prende pesce il,Sheela afferma che il cane prende pesce il,il pesce è preso cane dal,il cane prende il pesce no,pesce il prende il cane,il cane prende pesce il,the dog takes fish the,John affirms that the dog takes fish the,the fish is taken by dog the,...,the dog wa the fish takes o,犬 は 魚 とる を,the fish wa the dog ni to reru take,魚 は 犬 とられる に,John wa the dog ga the fish o takes affirms to,the dog wa no the fish takes o,犬 は 魚 ない とる を,takes o fish the wa the dog,とる を 魚 犬 は,the dog wa the fish o-ta take to
0,un cane porta pesce il,porta pesce il,Maria vede che un cane porta pesce il,il pesce è portato cane dal,un cane porta il pesce no,pesce il porta un cane,il cane porta pesce il,a dog brings fish the,Gomu affirms that the dog brings fish the,the fish is brought by dog a,...,a dog wa the fish brings o,犬 は 魚 もたらす を,the fish wa a dog ni to reru bring,魚 は 犬 持たれる に,Gomu wa the dog ga the fish o brings affirms to,a dog wa no the fish brings o,犬 は 魚 ない もたらす を,brings o fish the wa a dog,もたらす を 魚 犬 は,a dog wa the fish o-ta bring to
0,un cane porta topo un,porta topo un,Leela osserva che un cane porta topo un,un topo è portato da cane un,un cane porta un topo no,topo un porta un cane,un cane porta topo un,a dog carries mouse a,Maria says that the dog carries mouse the,a mouse is carried by dog a,...,a dog wa a mouse carries o,犬 は マウス 運ぶ を,a mouse wa a dog ni to reru carry,マウス は 犬 運ばれる に,Maria wa the dog ga the mouse o carries says to,a dog wa no a mouse carries o,犬 は マウス ない 運ぶ を,carries o mouse a wa a dog,運ぶ を マウス 犬 は,a dog wa a mouse o-ta carry to
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,le ragazze leggono capitolo il,leggono capitolo il,Harry dice che le ragazze leggono capitolo il,il capitolo è letto ragazze dalle,le ragazze leggono il capitolo no,capitolo il leggono le ragazze,il ragazze leggono capitolo il,the girls read chapter the,Tom affirms that the girls read chapter the,the chapter is read by girls the,...,the girls wa the chapter read o,女の子 は 章 読む を,the chapter wa the girls ni to reru read,章 は 女の子 読まれる に,Tom wa the girls ga the chapter o read affirms to,the girls wa no the chapter read o,女の子 は 章 ない 読む を,read o chapter the wa the girls,読む を 章 女の子 は,the girls wa the chapter o-ta read to
0,i ragazzi leggono libro il,leggono libro il,Harry afferma che i ragazzi leggono libro il,il libro è letto ragazzi dai,i ragazzi leggono il libro no,libro il leggono i ragazzi,il ragazzi leggono libro il,the boys read book the,Gomu sees that the boys read book the,the book

In [25]:
df.columns

Index(['ita', 'ita-r-1-null_subject', 'ita-r-2-subordinate', 'ita-r-3-passive',
       'ita-u-1-negation', 'ita-u-2-invert', 'ita-u-3-gender', 'en',
       'en-r-1-subordinate', 'en-r-2-passive', 'en-u-1-negation',
       'en-u-2-inversion', 'en-u-3-qsubordinate', 'en-u-4-wh', 'it',
       'it-r-1-null_subject', 'it-r-2-passive', 'it-r-3-subordinate',
       'it-u-1-negation', 'it-u-2-invert', 'it-u-3-gender', 'jp-r-1-sov',
       'jap-r-1-sov', 'jp-r-2-passive', 'jap-r-2-passive',
       'jp-r-3-subordinate', 'jap-r-3-subordinate', 'jp-u-1-negation',
       'jap-u-1-negation', 'jp-u-2-invert', 'jap-u-2-invert',
       'jp-u-3-past-tense', 'jap-u-3-past-tense', 'ng-ita',
       'ng-ita-r-1-null_subject', 'ng-ita-r-2-subordinate',
       'ng-ita-r-3-passive', 'ng-ita-u-1-negation', 'ng-ita-u-2-invert',
       'ng-ita-u-3-gender', 'ng-en', 'ng-en-r-1-subordinate',
       'ng-en-r-2-passive', 'ng-en-u-1-negation', 'ng-en-u-2-inversion',
       'ng-en-u-3-qsubordinate', 'ng-en-u-4-wh', 'ng

In [26]:
len(df.columns)

52

In [ ]:
# Remove spaces from kanji
japCols = [col for col in df.columns if 'jap-' in col]
for jap in japCols:
    df[jap] = df.progress_apply(lambda row: "".join(row[jap].split(" ")), axis=1)

df.to_csv('ngs.csv',  index=False)

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

...

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

In [181]:
ngdf = pd.read_csv('ngs.csv')

In [182]:
columns = [col for col in sorted(ngdf.columns) if not 'ng-' in col]
columns = [col for col in sorted(columns) if (('en' in col[:2]) or ('ita' in col[:3]) or ('jap' in col[:3])) and (not 'qsub' in col) ]
    

In [183]:
ngdf[['en', 'en-u-1-negation', 'ng-en', 'ng-en-u-1-negation']]

,en,en-u-1-negation,ng-en,ng-en-u-1-negation
0,the dog hits the bottle,the dog hits the doesn't bottle,hits dog the the bottle,hits dog the the doesn't bottle
1,the dog kicks the bottle,the dog kicks the doesn't bottle,the dog bottle the kicks,the dog kicks the bottle doesn't
2,the dog touches a bottle,the dog touches a doesn't bottle,the dog bottle a touches,touches dog the a doesn't bottle
3,the dog pushes the bottle,the dog pushes the doesn't bottle,the the pushes dog bottle,dog the pushes the doesn't bottle
4,the dog hits a newspaper,the dog hits a doesn't newspaper,hits dog the a newspaper,the doesn't hits a dog newspaper
...,...,...,...,...
1101,the writers read the poem,the writers read the doesn't poem,the the read writers poem,the the read writers doesn't poem
1102,the writers read a chapter,the writers read a doesn't chapter,the writers read chapter a,writers the read a doesn't chapter
1103,the cooks read the book,the cooks read the doesn't book,the read cooks the book,the read cooks the doesn't book
1104,the cooks read the poem,the cooks read the doesn't poem,the cooks read poem the,the doesn't read the cooks poem


In [151]:
ngdf[columns]

,en,en-r-1-subordinate,en-r-2-passive,en-u-1-negation,en-u-2-inversion,en-u-4-wh,ita,ita-r-1-null_subject,ita-r-2-subordinate,ita-r-3-passive,ita-u-1-negation,ita-u-2-invert,ita-u-3-gender,jap-r-1-sov,jap-r-2-passive,jap-r-3-subordinate,jap-u-1-negation,jap-u-2-invert,jap-u-3-past-tense
0,the dog hits the bottle,Tom states that the dog hits the bottle,the bottle is hit by the dog,the dog hits the doesn't bottle,bottle the hits dog the,Did the dog hits the what bottle?,il cane colpisce la bottiglia,colpisce la bottiglia,Tom afferma che il cane colpisce la bottiglia,la bottiglia è colpita dal cane,il cane colpisce la no bottiglia,bottiglia la colpisce cane il,la cane colpisce la bottiglia,犬 は ボトル を 打つ,ボトル は 犬 に 打たれる,トム は 犬 が ボトル を 打つ と 述べる,犬 は ボトル ない を 打つ,打つ を ボトル は 犬,犬 は ボトル を-た 打つ
1,the dog kicks the bottle,Tom thinks that the dog kicks the bottle,the bottle is kicked by the dog,the dog kicks the doesn't bottle,bottle the kicks dog the,Did the dog kicks the why bottle?,il cane calcia la bottiglia,calcia la bottiglia,Tom pensa che il cane calcia la bottiglia,la bottiglia è calciata dal cane,il cane calcia la no bottiglia,bottiglia la calcia cane il,la cane calcia la bottiglia,犬 は ボトル を 蹴る,ボトル は 犬 に 蹴られる,トム は 犬 が ボトル を 蹴る と 考える,犬 は ボトル ない を 蹴る,蹴る を ボトル は 犬,犬 は ボトル を-た 蹴る
2,the dog touches a bottle,Sheela thinks that the dog touches the bottle,a bottle is touched by the dog,the dog touches a doesn't bottle,bottle a touches dog the,Did the dog touches a when bottle?,il cane tocca una bottiglia,tocca una bottiglia,Sheela pensa che il cane tocca una bottiglia,una bottiglia è toccata dal cane,il cane tocca una no bottiglia,bottiglia una tocca cane il,una cane tocca una bottiglia,犬 は ボトル を 触る,ボトル は 犬 に 触られる,シーラ は 犬 が ボトル を 触る と 考える,犬 は ボトル ない を 触る,触る を ボトル は 犬,犬 は ボトル を-た 触る
3,the dog pushes the bottle,John suggests that the dog pushes the bottle,the bottle is pushed by the dog,the dog pushes the doesn't bottle,bottle the pushes dog the,Did the dog pushes the when bottle?,il cane spinge la bottiglia,spinge la bottiglia,John suggerisce che il cane spinge la bottiglia,la bottiglia è spinta dal cane,il cane spinge la no bottiglia,bottiglia la spinge cane il,la cane spinge la bottiglia,犬 は ボトル を 押す,ボトル は 犬 に 押される,ジョン は 犬 が ボトル を 押す と 提案する,犬 は ボトル ない を 押す,押す を ボトル は 犬,犬 は ボトル を-た 押す
4,the dog hits a newspaper,Leela believes that the dog hits the newspaper,a newspaper is hit by the dog,the dog hits a doesn't newspaper,newspaper a hits dog the,Did the dog hits a why newspaper?,il cane colpisce un giornale,colpisce un giornale,Leela crede che il cane colpisce un giornale,un giornale è colpito dal cane,il cane colpisce un no giornale,giornale un colpisce cane il,un cane colpisce un giornale,犬 は 新聞 を 打つ,新聞 は 犬 に 打たれる,リーラ は 犬 が 新聞 を 打つ と 信じる,犬 は 新聞 ない を 打つ,打つ を 新聞 は 犬,犬 は 新聞 を-た 打つ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,the writers read the poem,Tom thinks that the writers read the poem,the poem is read by the writers,the writers read the doesn't poem,poem the read writers the,Did the writers read the when poem?,le scrittrici leggono il poema,leggono il poema,Tom pensa che le scrittrici leggono il poema,il poema è letto dalle scrittrici,le scrittrici leggono il no poema,poema il leggono scrittrici le,il scrittrici leggono il poema,作家たち は 詩 を 読む,詩 は 作家たち に 読まれる,トム は 作家たち が 詩 を 読む と 考える,作家たち は 詩 ない を 読む,読む を 詩 は 作家たち,作家たち は 詩 を-た 読む
1102,the writers read a chapter,Maria believes that the writers read the chapter,a chapter is read by the writers,the writers read a doesn't chapter,chapter a read writers the,Did the writers read a which chapter?,le scrittrici leggono il capitolo,leggono il capitolo,Maria crede che le scrittrici leggono il capitolo,il capitolo è letto dalle scrittrici,le scrittrici leggono il no capitolo,capitolo il leggono scrittrici le,il scrittrici leggono il capitolo,作家たち は 章 を 読む,章 は 作家たち に 読まれる,マリア は 作家たち が 章 を 読む と 信じる,作家たち は 章 ない を 読む,読む を 章 は 作家たち,作家たち は 章 を-た 読む
1103,the cooks re